In [1]:
# Required package for running SQL queries: ipython-sql
# If not installed run: !pip install ipython-sql

# Load sql extention
%load_ext sql

import pandas as pd
import sqlite3

In [2]:
# Load cleaned data
df = pd.read_csv('./unwto_cleaned_long.csv')

# Connect to SQLIte database
conn = sqlite3.connect('tourism.db')


# Save dataframe to SQL table
df.to_sql('tourism_data', conn, if_exists='replace', index=False)

# Load sql extension with this connection
%sql sqlite:///tourism.db

In [3]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [4]:
# Data Exploration
%%sql
SELECT * FROM tourism_data LIMIT 5;

SyntaxError: invalid syntax (3229109774.py, line 3)

In [5]:
# Data 